In [ ]:
# This notebook is intended to demonstrate basic use of pyHARM.  It is a work in progress, additions or issues welcome
# Note this requires pyHARM to be installed via `setup.py develop` or `pip install .`, unless you move it to the root of this repository
import numpy as np
import matplotlib.pyplot as plt

import pyHARM
import pyHARM.ana.plot as pplt

# Suppress math warnings. These are just for /0 or sqrt(-)
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [ ]:
# The heart of pyHARM is the IharmDump object, which acts like a dictionary of different fluid variables, without storing them in memory as arrays.
# By default it just loads the primitive variables over the grid -- however, there are several options to add anything else present in HARM output.
# Very likely, you will want to cache the velocity and magnetic field 4-vectors with calc_derived=True
# You can also add the conserved variable forms (U vector) with calc_derived=True
# If they're present in the original file, you can also load information about the 4-current, floor hits, inversion failures, and B-field divergence -- see the function description of the IharmDump constructor __init__ in 'ana.iharm_dump', for which load_dump is an alias.

dump = pyHARM.load_dump("dump_00002000.h5", calc_derived=True)

In [ ]:
# Example of calculating the dimensionless EH magnetization, usually denoted \phi_B
# Note when you specify a variable for a reduction like this, 
# you can pass just the name of a variable directly, i.e. 'Pg' for gas pressure,
# or retrieve the variable, do some operations, and pass the resulting ndarray, as below.
# A full list of named variables is at the top of 'ana.variables', and reductions like 'shell_sum' are defined in 'ana.reductions'
# A limited library of prefixes, indices, etc is also supported; documentation soon, see 'ana.iharm_dump' in the meantime
0.5 * pyHARM.shell_sum(dump, np.fabs(dump['B1']), at_zone=5) / np.sqrt(-pyHARM.shell_sum(dump, 'FM', at_zone=5))

In [ ]:
# Check the accretion rate vs radius, summed only in the disk and over the full sphere
# The function 'pretty' tries to guess the LaTeX name that represents a variable.
# It's necessarily not perfect -- mostly it's useful when writing code meant to plot any available variable, such as 'auto_plot.py' or 'quick_plot.py' in scripts/
plt.plot(dump['r'][:,0,0], -pyHARM.shell_sum(dump, 'FM', th_slice=(5*np.pi/12, 7*np.pi/12)), label="Disk")
plt.plot(dump['r'][:,0,0], -pyHARM.shell_sum(dump, 'FM'), label="All")
plt.xlim(0,50)
plt.ylim(0,30)
plt.xlabel(pyHARM.pretty('r'))
plt.ylabel(pyHARM.pretty('Mdot'))
plt.title(r"Shell-summed inward mass flux by radius")
plt.legend()
plt.show()

In [ ]:
# The shell-averaged azimuthal velocity as a function of r is usually more stable
plt.plot(dump['r'][:,0,0], pyHARM.shell_avg(dump, 'u^phi'))
plt.xlim(0,400)
plt.yscale('log')
plt.xlabel(pyHARM.pretty('r'))
plt.ylabel(pyHARM.pretty('u^phi'))
plt.title(r"Shell-average u^{\phi}")
plt.show()

In [ ]:
# Is pi/3-2pi/3 really representative of the disk?  Let's check how much of the accretion
# falls in that range
# Note this uses pyHARM's 2D plotting functions in 'ana.plot'
fig, ax = plt.subplots(1,1)
pplt.plot_xz(ax, dump, np.log10(-dump['FM']), window=[-5,5,-5,5])
pplt.overlay_contours(ax, dump, 'th', [np.pi/3, 2*np.pi/3])
plt.show()

In [ ]:
# Histogram the log10 of density, weighted by area.
# This demonstrates some of the prefixes that can be used to request particular operations on variables
# Perhaps the most common is 'log_' which simply avoids having to write np.log10() everywhere
# Note this little "language" isn't set in stone yet -- in particular:
# 1. there may be differences in how plotting vs IharmDump handle 'log_'
# 2. Future reductions may be added in the same way as pdf_, or it may be removed in favor of pyHARM.pdf()
# Note that the 'bins' returned represents bin *borders*, as in np.hist -- for this informal plot, we just use the start of each bin
d_var, d_var_bins = dump['pdf_log_rho']
plt.plot(d_var_bins[:-1], d_var)

In [ ]:
# TODO more 2D plotting, more exotic variables.  Results files & plotting, eventually. Units, parallelization?